# Real-Time Multilingual ASR with Whisper

This notebook implements a production-ready, real-time speech recognition system using OpenAI's Whisper models.

**Features:**
- Data preparation with augmentation
- Model fine-tuning with multiple variants
- Comprehensive evaluation (WER, CER, latency)
- Real-time streaming inference
- Full MLOps best practices (versioning, logging, reproducibility)

**Author:** COMP3057 Project 
**Version:** 1.0.0

## 1. Setup & Installation

### Colab Configuration Guide

**Resource Constraints:** A100 GPU (~40GB), 220GB Disk

**Quick Start Options:**

| Profile | Model | Samples | Epochs | Time | Disk | Quality |
|---------|-------|---------|--------|------|------|---------|
| **Fast Demo** | tiny | 50/10 | 1 | ~5min | ~2GB | Basic |
| **Balanced** | base | 200/40 | 2 | ~20min | ~5GB | Good |
| **Best Quality** | small | 500/100 | 3 | ~60min | ~10GB | Better |

**Adjustable Parameters:**
- `TRAIN_SAMPLES` / `VAL_SAMPLES` - Dataset size
- `MODEL_VARIANT` - Model quality (tiny/base/small)
- `TRAIN_EPOCHS` - Training duration

**Tips:**
- Start with Fast Demo to verify everything works
- Increase resources gradually if you have time
- Checkpoints auto-save to Google Drive

In [ ]:
# Setup environment and clone project
import os
import sys

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running in Google Colab")
    
    # Mount Google Drive for saving checkpoints and logs
    from google.colab import drive
    if not os.path.exists('/content/drive'):
        print("Mounting Google Drive...")
        drive.mount('/content/drive', force_remount=False)
        print("Drive mounted at /content/drive")
    else:
        print("Drive already mounted")
    
    # --- Project Setup with Absolute Paths (Prevents Nesting) ---
    PROJECT_DIR = '/content/COMP3057_Project'  # Use ABSOLUTE path
 
    # Always start from /content to avoid nested cloning
    os.chdir('/content')
    print(f"Working from: {os.getcwd()}")
    
    # Check for nested structure and fix it
    if os.path.exists('COMP3057_Project/COMP3057_Project'):
        print("\nNested structure detected. Fixing...")
        import shutil
        
        # Check if outer is just a wrapper
        outer_contents = os.listdir('COMP3057_Project')
        if len(outer_contents) == 1 and outer_contents[0] == 'COMP3057_Project':
            print("   Moving inner project to correct location...")
            shutil.move('COMP3057_Project/COMP3057_Project', 'COMP3057_Project_temp')
            shutil.rmtree('COMP3057_Project')
            shutil.move('COMP3057_Project_temp', 'COMP3057_Project')
            print("   Fixed nested structure")
        else:
            print("   Complex nesting - keeping as is")
 
    # Clone repository if not exists
    if not os.path.exists(PROJECT_DIR):
        print("\nCloning repository from GitHub...")
        !git clone https://github.com/jimmy00415/COMP3057_Project.git
        print("Repository cloned")
    else:
        print("Repository already exists")
        
        # Pull latest changes if already exists
        print("   Checking for updates...")
        os.chdir(PROJECT_DIR)
        !git pull
        os.chdir('/content')
    
    # Change to project directory
    os.chdir(PROJECT_DIR)
    print(f"Working directory: {os.getcwd()}")

    # Add project root to Python path
    if os.getcwd() not in sys.path:
        sys.path.insert(0, os.getcwd())
        print(f"Added '{os.getcwd()}' to Python path")
 
    # Verify structure
    expected_files = ['src', 'config.yaml', 'requirements.txt', 'whisper_asr_colab.ipynb']
    found_files = [f for f in expected_files if os.path.exists(f)]
    print(f"Structure check: {len(found_files)}/{len(expected_files)} key items found")
    
    if len(found_files) < len(expected_files):
        missing = set(expected_files) - set(found_files)
        print(f"   Missing: {missing}")
    
    # Install dependencies
    print("\nInstalling dependencies...")
    !pip install -q -r requirements.txt
    !pip install -q sounddevice
    print("Dependencies installed")

else:
    print("Running locally")
    print(f"Working directory: {os.getcwd()}")

# Check GPU availability
import torch
gpu_available = torch.cuda.is_available()
if gpu_available:
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"\nGPU: {gpu_name}")
    print(f"   Memory: {gpu_memory:.1f} GB")
    print(f"   CUDA Version: {torch.version.cuda}")
else:
    print("\nWARNING: No GPU detected. Training will be very slow.")
    print("   Enable GPU: Runtime -> Change runtime type -> GPU (T4)")

print("\n" + "="*60)
print("SETUP COMPLETE")
print("="*60)
print("\nNext steps for post-training evaluation:")
print("  1. Run the Quick Restart cell (Section 1) to load your trained model")
print("  2. Run the validation dataset loader (Section 1)")
print("  3. Skip to Section 7 (Model Evaluation)")
print("="*60)

### Quick Restart (Skip Training)

**If you've already trained a model and just want to run inference/evaluation:**

1. Run the Setup cell above (mounts Drive, clones repo, installs deps)
2. Run the Quick Restart cell below to load your trained model
3. Skip to Section 7 (Evaluation) or later sections

This avoids re-running the expensive training steps.

In [ ]:
# Quick Restart - Load existing model without retraining
# Use this to skip training and jump to evaluation
# Run after the setup cell if you have a trained model

import sys
import os

print("="*60)
print("QUICK RESTART - POST-TRAINING EVALUATION")
print("="*60)

# Ensure project path is available
if 'google.colab' in sys.modules:
    project_root = '/content/COMP3057_Project'
    if os.path.exists(project_root):
        os.chdir(project_root)
        if project_root not in sys.path:
            sys.path.insert(0, project_root)
        print(f"Working in {project_root}")
    else:
        raise RuntimeError("Project not found. Run the Setup cell first.")
else:
    # Local environment
    project_root = os.getcwd()
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
    print(f"Working in {project_root}")

# Import all necessary modules
print("\nImporting modules...")
import torch
import numpy as np

from src.utils.config import load_config
from src.models.whisper_model import WhisperModelManager
from src.utils.versioning import ModelRegistry

# Import evaluation and inference modules
from src.evaluation import (
 ModelEvaluator,
 LatencyBenchmark,
 EvaluationVisualizer,
 generate_comparison_table
)

from src.inference import (
 StreamingASR,
 BatchInference
)

from src.data import VoiceActivityDetector

print("Modules imported")

# Load config
config = load_config('config.yaml')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# Initialize VAD for streaming
vad = VoiceActivityDetector(threshold=config['data']['vad_threshold'])

# Auto-detect checkpoint and model variant
print("\nSearching for trained model...")

# Priority order: best_model_hf > final_model > checkpoint-*/
checkpoint_candidates = [
 "checkpoints/best_model_hf",
 "final_model",
 "checkpoints/checkpoint-final",
]

# Also check Google Drive if in Colab
if 'google.colab' in sys.modules:
 drive_checkpoints = [
 "/content/drive/MyDrive/COMP3057_ASR/checkpoints/best_model_hf",
 "/content/drive/MyDrive/COMP3057_ASR/final_model",
 ]
 checkpoint_candidates = drive_checkpoints + checkpoint_candidates

# Find the first existing checkpoint
CHECKPOINT_PATH = None
for candidate in checkpoint_candidates:
 if os.path.exists(candidate):
 CHECKPOINT_PATH = candidate
 print(f"Found checkpoint: {CHECKPOINT_PATH}")
 break

if CHECKPOINT_PATH is None:
 print("No trained checkpoint found.")
 print(" Checked locations:")
 for candidate in checkpoint_candidates:
 print(f" - {candidate}")
 
 # Fallback to base model
 print("\n Loading base model instead (not fine-tuned)...")
 MODEL_VARIANT = config.get('model', {}).get('variant', 'small')
 model_manager = WhisperModelManager(config)
 model, processor = model_manager.initialize_model(variant=MODEL_VARIANT, device=str(device))
 print(f"Loaded base {MODEL_VARIANT} model (NOT fine-tuned)")
 
else:
 # Load from checkpoint
 print(f"\nLoading fine-tuned model from: {CHECKPOINT_PATH}")
 from transformers import WhisperForConditionalGeneration, WhisperProcessor
 
 model = WhisperForConditionalGeneration.from_pretrained(CHECKPOINT_PATH)
 processor = WhisperProcessor.from_pretrained(CHECKPOINT_PATH)
 model = model.to(device)
 
 # Detect model variant from config
 from transformers import WhisperConfig
 model_config = model.config
 if hasattr(model_config, 'd_model'):
 d_model = model_config.d_model
 variant_map = {256: 'tiny', 512: 'base', 768: 'small', 1024: 'medium', 1280: 'large'}
 MODEL_VARIANT = variant_map.get(d_model, 'small')
 else:
 MODEL_VARIANT = 'small'
 
 print(f"Loaded fine-tuned model (variant: {MODEL_VARIANT})")

# Initialize model registry
model_registry = ModelRegistry()

# Set model to eval mode
model.eval()

# Get model info (needed for visualization)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

model_info = {
 'total_parameters': total_params,
 'trainable_parameters': trainable_params,
 'variant': MODEL_VARIANT
}

# Initialize batch inference (needed for streaming test)
batch_inference = BatchInference(
 model=model,
 processor=processor,
 device=str(device),
 batch_size=config['inference']['batch_size']
)

print("\n" + "="*60)
print("QUICK RESTART COMPLETE")
print("="*60)
print(f"Model Summary:")
print(f" - Variant: {MODEL_VARIANT}")
print(f" - Parameters: {total_params:,}")
print(f" - Device: {device}")
print(f" - Status: {'Fine-tuned' if CHECKPOINT_PATH else 'Base model (not trained)'}")
print(f" - Checkpoint: {CHECKPOINT_PATH if CHECKPOINT_PATH else 'N/A'}")
print("\nReady for evaluation.")
print(" Next: Run the validation dataset loader cell below")
print("="*60)

In [ ]:
# Load validation dataset for evaluation
# This loads a small validation set without needing the full training pipeline

from datasets import load_dataset, Audio
import soundfile as sf
import io
import numpy as np

VAL_SAMPLES = 400  # Match training configuration

print(f"Loading validation dataset ({VAL_SAMPLES} samples)...")

    # Try minds14 first
    print("Loading minds14 dataset...")
    dataset = load_dataset("PolyAI/minds14", "en-US", split=f"train[:{VAL_SAMPLES}]")
    dataset = dataset.cast_column("audio", Audio(decode=False))
    
    def decode_audio(example):
        audio = example["audio"]
        if "bytes" in audio and audio["bytes"] is not None:
            waveform, sr = sf.read(io.BytesIO(audio["bytes"]))
        elif "path" in audio and audio["path"] is not None:
            waveform, sr = sf.read(audio["path"])
        else:
            waveform, sr = np.array([], dtype=np.float32), 16000
        
        if not isinstance(waveform, np.ndarray):
            waveform = np.array(waveform, dtype=np.float32)
        else:
            waveform = waveform.astype(np.float32)
        
        example["audio_decoded"] = {"array": waveform.tolist(), "sampling_rate": int(sr)}
        return example
 return example
    dataset = dataset.map(decode_audio, desc="Decoding audio")
    dataset = dataset.remove_columns(["audio"])
    dataset = dataset.rename_column("audio_decoded", "audio")
    val_dataset = dataset
    dataset_name = "minds14_en"
 dataset_name = "minds14_en"
 
    print(f"minds14 failed: {e}, trying LibriSpeech...")
    dataset = load_dataset("openslr/librispeech_asr", "clean", split=f"test[:{VAL_SAMPLES}]")
    dataset = dataset.cast_column("audio", Audio(decode=False))
    dataset = dataset.map(decode_audio, desc="Decoding audio")
    dataset = dataset.remove_columns(["audio"])
    dataset = dataset.rename_column("audio_decoded", "audio")
    val_dataset = dataset
    dataset_name = "librispeech_clean"
 dataset_name = "librispeech_clean"

# Detect text column - Make it global for use in other cells
text_column = None
    if col in val_dataset.column_names:
        text_column = col
        break
 break

    raise ValueError(f"Could not find text column. Available: {val_dataset.column_names}")
 raise ValueError(f"Could not find text column. Available: {val_dataset.column_names}")

print(f"  Samples: {len(val_dataset)}")
print(f"  Text column: '{text_column}'")
print(f" Text column: '{text_column}'")

# Create validation data loader
from src.data import WhisperDataset, DataCollatorWithPadding
from torch.utils.data import DataLoader

    val_dataset,
    processor,
    audio_column="audio",
    text_column=text_column,
    max_audio_length_sec=config['data']['audio_max_length_sec'],
    augmenter=None
 augmenter=None
)

collator = DataCollatorWithPadding(processor)
    val_dataset_wrapper,
    batch_size=config['training']['batch_size'],
    shuffle=False,
    collate_fn=collator,
    num_workers=0
 num_workers=0
)

print(f"Data loader created ({len(val_loader)} batches)")
print(f"\nDataset ready for evaluation.")
print(f" Next: Skip to Section 7 (Model Evaluation)")

In [ ]:
# POST-TRAINING VALIDATION - Verify all components are ready
print("=" * 80)
print("POST-TRAINING VALIDATION CHECK")
print("=" * 80)

# Track validation results
validation_passed = True
required_vars = {}

# Check 1: Model and processor
print("\n1. Model & Processor:")
try:
 assert 'model' in dir(), "model not found"
 assert 'processor' in dir(), "processor not found"
 assert model is not None, "model is None"
 assert processor is not None, "processor is None"
 print(f" Model: {type(model).__name__}")
 print(f" Processor: {type(processor).__name__}")
 required_vars['model'] = True
 required_vars['processor'] = True
except AssertionError as e:
 print(f" FAILED: {e}")
 validation_passed = False
 required_vars['model'] = False
 required_vars['processor'] = False

# Check 2: Device
print("\n2. Device Configuration:")
try:
 assert 'device' in dir(), "device not found"
 print(f" Device: {device}")
 print(f" Model on device: {next(model.parameters()).device}")
 required_vars['device'] = True
except AssertionError as e:
 print(f" FAILED: {e}")
 validation_passed = False
 required_vars['device'] = False

# Check 3: Config
print("\n3. Configuration:")
try:
 assert 'config' in dir(), "config not found"
 assert config is not None, "config is None"
 print(f" Config loaded")
 print(f" Inference batch size: {config['inference']['batch_size']}")
 required_vars['config'] = True
except AssertionError as e:
 print(f" FAILED: {e}")
 validation_passed = False
 required_vars['config'] = False

# Check 4: Validation dataset
print("\n4. Validation Dataset:")
try:
 assert 'val_dataset' in dir(), "val_dataset not found"
 assert 'val_loader' in dir(), "val_loader not found"
 assert 'text_column' in dir(), "text_column not found"
 print(f" Dataset: {len(val_dataset)} samples")
 print(f" Data loader: {len(val_loader)} batches")
 print(f" Text column: '{text_column}'")
 required_vars['val_dataset'] = True
 required_vars['val_loader'] = True
 required_vars['text_column'] = True
except AssertionError as e:
 print(f" FAILED: {e}")
 print(f" Action: Run the validation dataset loader cell above")
 validation_passed = False
 required_vars['val_dataset'] = False
 required_vars['val_loader'] = False
 required_vars['text_column'] = False

# Check 5: Model info
print("\n5. Model Information:")
try:
 assert 'model_info' in dir(), "model_info not found"
 assert 'MODEL_VARIANT' in dir(), "MODEL_VARIANT not found"
 print(f" Variant: {MODEL_VARIANT}")
 print(f" Parameters: {model_info['total_parameters']:,}")
 required_vars['model_info'] = True
 required_vars['MODEL_VARIANT'] = True
except AssertionError as e:
 print(f" FAILED: {e}")
 print(f" Action: Run the Quick Restart cell")
 validation_passed = False
 required_vars['model_info'] = False
 required_vars['MODEL_VARIANT'] = False

# Check 6: Inference components
print("\n6. Inference Components:")
try:
 assert 'batch_inference' in dir(), "batch_inference not found"
 assert 'vad' in dir(), "vad not found"
 print(f" BatchInference initialized")
 print(f" VAD initialized")
 required_vars['batch_inference'] = True
 required_vars['vad'] = True
except AssertionError as e:
 print(f" FAILED: {e}")
 print(f" Action: Run the Quick Restart cell")
 validation_passed = False
 required_vars['batch_inference'] = False
 required_vars['vad'] = False

# Check 7: Test forward pass
print("\n7. Forward Pass Test:")
try:
 test_sample = val_dataset[0]
 test_audio = torch.tensor(test_sample['audio']['array'])
 
 model.eval()
 with torch.no_grad():
 input_features = processor(
 test_audio,
 sampling_rate=16000,
 return_tensors="pt"
 ).input_features.to(device)
 
 predicted_ids = model.generate(input_features, max_length=50)
 test_transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
 
 print(f" Forward pass successful")
 print(f" Test transcription: '{test_transcription[:50]}...'")
 required_vars['forward_pass'] = True
except Exception as e:
 print(f" FAILED: {e}")
 validation_passed = False
 required_vars['forward_pass'] = False

# Summary
print("\n" + "=" * 80)
if validation_passed:
 print("ALL CHECKS PASSED - READY FOR EVALUATION")
 print("=" * 80)
 print("\nYou can now run:")
 print(" - Section 7: Model Evaluation (WER/CER/Latency)")
 print(" - Section 8: Streaming Inference Test")
 print(" - Section 10: Visualization")
else:
 print("VALIDATION FAILED - MISSING COMPONENTS")
 print("=" * 80)
 print("\nRequired actions:")
 if not required_vars.get('model') or not required_vars.get('processor'):
 print(" 1. Run the Quick Restart cell (Section 1)")
 if not required_vars.get('val_dataset'):
 print(" 2. Run the validation dataset loader cell")
 if not required_vars.get('batch_inference'):
 print(" 3. Re-run the Quick Restart cell")
 
 print("\nMissing variables:")
 for var, status in required_vars.items():
 if not status:
 print(f" - {var}")

print("=" * 80)

### Professional-Grade Improvements

This notebook includes production-grade enhancements:
- **Error Recovery:** Automatic handling of corrupted data, NaN/Inf, OOM errors
- **Memory Management:** Intelligent cleanup and optimization
- **Audio Validation:** Quality checks for all audio samples
- **Structured Logging:** Comprehensive metrics tracking
- **Safe Checkpointing:** Atomic writes with validation

See `IMPROVEMENTS.md` for full details.

In [ ]:
# Colab Resource Optimization & Monitoring
import shutil
import sys

# Re-check if in Colab (in case this cell is run independently)
IN_COLAB = 'google.colab' in sys.modules

def check_disk_space():
 """Check available disk space."""
 total, used, free = shutil.disk_usage("/")
 print(f"ð¾ Disk Space:")
 print(f" Total: {total // (2**30)} GB")
 print(f" Used: {used // (2**30)} GB")
 print(f" Free: {free // (2**30)} GB")
 return free // (2**30)

def check_gpu_memory():
 """Check GPU memory usage."""
 import torch
 if torch.cuda.is_available():
 allocated = torch.cuda.memory_allocated(0) / 1e9
 reserved = torch.cuda.memory_reserved(0) / 1e9
 total = torch.cuda.get_device_properties(0).total_memory / 1e9
 print(f"\nð® GPU Memory:")
 print(f" Allocated: {allocated:.2f} GB")
 print(f" Reserved: {reserved:.2f} GB")
 print(f" Total: {total:.2f} GB")
 print(f" Free: {total - reserved:.2f} GB")

def cleanup_cache():
 """Clear unnecessary cache to free disk space."""
 import torch
 if torch.cuda.is_available():
 torch.cuda.empty_cache()
 import gc
 gc.collect()
 print("â?Cache cleared")

if IN_COLAB:
 # Setup Google Drive paths for checkpoints
 DRIVE_ROOT = '/content/drive/MyDrive/COMP3057_ASR'
 import os
 os.makedirs(DRIVE_ROOT, exist_ok=True)
 os.makedirs(f'{DRIVE_ROOT}/checkpoints', exist_ok=True)
 os.makedirs(f'{DRIVE_ROOT}/logs', exist_ok=True)
 print(f"\nð Google Drive storage: {DRIVE_ROOT}")
 
 # Create symlinks to save to Drive instead of local disk
 if os.path.exists('checkpoints') and not os.path.islink('checkpoints'):
 shutil.rmtree('checkpoints')
 if not os.path.exists('checkpoints'):
 os.symlink(f'{DRIVE_ROOT}/checkpoints', 'checkpoints')
 print("â?Checkpoints will be saved to Google Drive")
 
 if os.path.exists('logs') and not os.path.islink('logs'):
 shutil.rmtree('logs')
 if not os.path.exists('logs'):
 os.symlink(f'{DRIVE_ROOT}/logs', 'logs')
 print("â?Logs will be saved to Google Drive")
 
 # Check initial resources
 print("\nð Initial Resource Check:")
 free_disk = check_disk_space()
 check_gpu_memory()
 
 if free_disk < 50:
 print("\nâ ï¸ WARNING: Low disk space! Consider:")
 print(" 1. Use smaller dataset (already optimized)")
 print(" 2. Use 'tiny' or 'base' model variant")
 print(" 3. Reduce save_steps to save fewer checkpoints")
else:
 # Define cleanup_cache for local use
 import torch
 def cleanup_cache():
 if torch.cuda.is_available():
 torch.cuda.empty_cache()
 import gc
 gc.collect()

In [ ]:
# Import project modules
import torch
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display

# Import project modules
from src.utils import (
 load_config,
 set_seed,
 setup_logging,
 get_device,
 ExperimentLogger,
 DataVersionManager,
 ModelRegistry
)

from src.data import (
 AudioPreprocessor,
 VoiceActivityDetector,
 AudioAugmenter,
 WhisperDataset,
 prepare_datasets,
 create_dataloaders
)

from src.models import (
 WhisperModelManager,
 compare_models
)

from src.training import WhisperTrainer

from src.evaluation import (
 ModelEvaluator,
 LatencyBenchmark,
 TrainingVisualizer,
 EvaluationVisualizer,
 generate_comparison_table
)

from src.inference import (
 StreamingASR,
 BatchInference
)

print("â?All modules imported successfully")

## 2. Configuration & Reproducibility Setup

In [ ]:
# Load configuration
config = load_config('config.yaml')

# Set random seeds for reproducibility
set_seed(config['project']['seed'])

# Setup logging
logger = setup_logging('INFO', 'logs/training.log')

# Get device
device = get_device(config['project']['device'])
logger.info(f"Using device: {device}")

# Initialize experiment tracking (choose: wandb, mlflow, or tensorboard)
experiment_logger = ExperimentLogger(
 backend=config['mlops']['experiment_tracking']['backend'],
 project_name=config['mlops']['experiment_tracking']['project_name'],
 config=config
)

# Initialize versioning
data_version_manager = DataVersionManager(
 config['mlops']['versioning']['data_version_file']
)
model_registry = ModelRegistry(
 config['mlops']['versioning']['model_registry']
)

print(f"â?Configuration loaded")
print(f" - Seed: {config['project']['seed']}")
print(f" - Device: {device}")
print(f" - Tracking: {config['mlops']['experiment_tracking']['backend']}")

## 3. Data Preparation

In [ ]:
# Initialize preprocessing utilities
audio_preprocessor = AudioPreprocessor(
 target_sr=config['data']['sampling_rate'],
 normalize=True
)

vad = VoiceActivityDetector(
 threshold=config['data']['vad_threshold']
)

augmenter = AudioAugmenter(
 speed_perturbation=config['data']['augmentation']['speed_perturbation'],
 pitch_shift_semitones=config['data']['augmentation']['pitch_shift_semitones'],
 background_noise_prob=config['data']['augmentation']['background_noise_prob']
) if config['data']['augmentation']['enabled'] else None

print("â?Preprocessing utilities initialized")

In [ ]:
# Load datasets
# Using small subsets optimized for Colab's disk/memory constraints
# For production with more resources, increase the sample counts

from datasets import load_dataset, Audio
import soundfile as sf
import io
import numpy as np

# Configuration for Colab
TRAIN_SAMPLES = 2000 # Increased for better model quality (80GB GPU can handle this)
VAL_SAMPLES = 400 # Proportional validation set for robust evaluation

print(f"Loading dataset with {TRAIN_SAMPLES} train + {VAL_SAMPLES} val samples...")
print("(Optimized for Colab - increase samples for production)")

try:
 # Try minds14 first - smaller and faster to download
 print("\nTrying minds14 dataset (lightweight, ~50MB)...")
 dataset = load_dataset(
 "PolyAI/minds14",
 "en-US",
 split=f"train[:{TRAIN_SAMPLES + VAL_SAMPLES}]"
 )
 
 # Cast audio column to disable automatic decoding
 dataset = dataset.cast_column("audio", Audio(decode=False))
 
 # Pre-decode all audio using soundfile - store in separate column
 def decode_audio(example):
 audio = example["audio"]
 if "bytes" in audio and audio["bytes"] is not None:
 waveform, sr = sf.read(io.BytesIO(audio["bytes"]))
 elif "path" in audio and audio["path"] is not None:
 waveform, sr = sf.read(audio["path"])
 else:
 waveform, sr = np.array([], dtype=np.float32), 16000
 
 # Ensure numpy array
 if not isinstance(waveform, np.ndarray):
 waveform = np.array(waveform, dtype=np.float32)
 else:
 waveform = waveform.astype(np.float32)
 
 # Store decoded audio in NEW column
 example["audio_decoded"] = {"array": waveform.tolist(), "sampling_rate": int(sr)}
 return example
 
 print("Pre-decoding audio samples...")
 dataset = dataset.map(decode_audio, desc="Decoding audio")
 
 # Remove original audio column with Audio feature, rename decoded column
 dataset = dataset.remove_columns(["audio"])
 dataset = dataset.rename_column("audio_decoded", "audio")
 
 # Split into train/val
 split_point = TRAIN_SAMPLES
 train_dataset = dataset.select(range(split_point))
 val_dataset = dataset.select(range(split_point, TRAIN_SAMPLES + VAL_SAMPLES))
 
 dataset_name = "minds14_en"
 print(f"â?minds14 loaded successfully")
 
except Exception as e:
 print(f"minds14 failed: {e}")
 print("\nTrying LibriSpeech (larger, ~300MB)...")
 
 # Fallback to LibriSpeech
 dataset = load_dataset(
 "openslr/librispeech_asr",
 "clean",
 split=f"test[:{TRAIN_SAMPLES + VAL_SAMPLES}]"
 )
 
 # Cast audio column to disable automatic decoding
 dataset = dataset.cast_column("audio", Audio(decode=False))
 
 # Pre-decode all audio using soundfile
 def decode_audio(example):
 audio = example["audio"]
 if "bytes" in audio and audio["bytes"] is not None:
 waveform, sr = sf.read(io.BytesIO(audio["bytes"]))
 elif "path" in audio and audio["path"] is not None:
 waveform, sr = sf.read(audio["path"])
 else:
 waveform, sr = np.array([], dtype=np.float32), 16000
 
 if not isinstance(waveform, np.ndarray):
 waveform = np.array(waveform, dtype=np.float32)
 else:
 waveform = waveform.astype(np.float32)
 
 example["audio_decoded"] = {"array": waveform.tolist(), "sampling_rate": int(sr)}
 return example
 
 print("Pre-decoding audio samples...")
 dataset = dataset.map(decode_audio, desc="Decoding audio")
 
 # Remove original audio column, rename decoded
 dataset = dataset.remove_columns(["audio"])
 dataset = dataset.rename_column("audio_decoded", "audio")
 
 # Split into train/val
 split_point = TRAIN_SAMPLES
 train_dataset = dataset.select(range(split_point))
 val_dataset = dataset.select(range(split_point, TRAIN_SAMPLES + VAL_SAMPLES))
 
 dataset_name = "librispeech_clean"
 print(f"â?LibriSpeech loaded")

print(f"\nâ?Dataset loaded: {dataset_name}")
print(f" - Training samples: {len(train_dataset)}")
print(f" - Validation samples: {len(val_dataset)}")

# Inspect dataset structure
print(f"\nð Dataset structure:")
print(f" - Columns: {train_dataset.column_names}")

# Log dataset version
data_version_manager.log_dataset_version(
 dataset_name=dataset_name,
 version=f"colab_demo_{TRAIN_SAMPLES}train_{VAL_SAMPLES}val",
 metadata={'train': len(train_dataset), 'val': len(val_dataset)}
)

# Check disk space after loading
if IN_COLAB:
 print("\nð Disk space after dataset load:")
 check_disk_space()


## 4. Model Initialization

In [ ]:
# Initialize model manager
model_manager = WhisperModelManager(config)

# Choose model variant based on Colab resources
# Recommendations for A100 (40GB):
# - tiny: 39M params, ~500MB, fastest (RECOMMENDED for demo)
# - base: 74M params, ~1GB, good balance
# - small: 244M params, ~2GB, better accuracy
# - medium: 769M params, ~6GB, best accuracy (may be slow)

MODEL_VARIANT = 'small' # Change to 'base' or 'small' if you have time/resources

print(f"ð¤ Loading Whisper model: {MODEL_VARIANT}")
print(f" (Optimized for Colab - use tiny/base for best experience)")

# Load model and processor
model, processor = model_manager.initialize_model(
 variant=MODEL_VARIANT,
 device=str(device)
)

# Get model info
model_info = model_manager.get_model_info()
print(f"\nâ?Model initialized: {MODEL_VARIANT}")
print(f" - Total parameters: {model_info['total_parameters']:,}")
print(f" - Trainable parameters: {model_info['trainable_parameters']:,}")

# Check GPU memory after model load
if IN_COLAB:
 check_gpu_memory()

In [ ]:
# Compare model variants
variants_info = compare_models(config)

print("\nWhisper Model Variants Comparison:")
print("-" * 60)
for variant, info in variants_info.items():
 print(f"{variant:10s} | Params: {info['params']:8s} | Speed: {info['speed']:10s} | Accuracy: {info['accuracy']}")

## 5. Prepare Data Loaders

In [ ]:
# Create PyTorch datasets
# Auto-detect column names from the dataset
audio_column = "audio" # Standard across most datasets
# Text column varies: "text", "sentence", "transcription", etc.
text_column = None
for col in ["text", "sentence", "transcription", "transcript"]:
 if col in train_dataset.column_names:
 text_column = col
 break

if text_column is None:
 raise ValueError(f"Could not find text column. Available columns: {train_dataset.column_names}")

print(f"Using columns: audio='{audio_column}', text='{text_column}'")

train_dataset_wrapper = WhisperDataset(
 train_dataset,
 processor,
 audio_column=audio_column,
 text_column=text_column,
 max_audio_length_sec=config['data']['audio_max_length_sec'],
 augmenter=augmenter # Apply augmentation only to training
)

val_dataset_wrapper = WhisperDataset(
 val_dataset,
 processor,
 audio_column=audio_column,
 text_column=text_column,
 max_audio_length_sec=config['data']['audio_max_length_sec'],
 augmenter=None # No augmentation for validation
)

# Create data loaders
from src.data import DataCollatorWithPadding
from torch.utils.data import DataLoader

collator = DataCollatorWithPadding(processor)

train_loader = DataLoader(
 train_dataset_wrapper,
 batch_size=config['training']['batch_size'],
 shuffle=True,
 collate_fn=collator,
 num_workers=0
)

val_loader = DataLoader(
 val_dataset_wrapper,
 batch_size=config['training']['batch_size'],
 shuffle=False,
 collate_fn=collator,
 num_workers=0
)

print(f"\nâ?Data loaders created")
print(f" - Training batches: {len(train_loader)}")
print(f" - Validation batches: {len(val_loader)}")

## 6. Model Fine-Tuning

In [ ]:
# Prepare model for training
model = model_manager.prepare_for_training()

# Initialize trainer
trainer = WhisperTrainer(
 model=model,
 processor=processor,
 train_loader=train_loader,
 val_loader=val_loader,
 config=config,
 device=str(device),
 experiment_logger=experiment_logger
)

print("â?Trainer initialized")

In [ ]:
# Validation Test - Run before training
print("=" * 60)
print("PRE-TRAINING VALIDATION")
print("=" * 60)

# Test 1: Check config
print("\n1. Configuration:")
print(f" Learning rate: {config['training']['learning_rate']}")
print(f" Batch size: {config['training']['batch_size']}")
print(f" Grad accumulation: {config['training']['gradient_accumulation_steps']}")
print(f" Augmentation: {config['data']['augmentation']['enabled']}")
assert config['training']['learning_rate'] <= 1e-4, "LR too high!"
assert not config['data']['augmentation']['enabled'], "Augmentation should be disabled!"
print(" Config safe")

# Test 2: Sample batch
print("\n2. Testing sample batch:")
test_batch = next(iter(train_loader))
print(f" Input shape: {test_batch['input_features'].shape}")
print(f" Labels shape: {test_batch['labels'].shape}")

# Check for NaN/Inf
has_nan = torch.isnan(test_batch['input_features']).any()
has_inf = torch.isinf(test_batch['input_features']).any()
assert not has_nan, "NaN detected in features!"
assert not has_inf, "Inf detected in features!"
print(" No NaN/Inf in batch")

# Test 3: Forward pass
print("\n3. Testing forward pass:")
model.eval()
with torch.no_grad():
 test_out = model(
 input_features=test_batch['input_features'][:2].to(device),
 labels=test_batch['labels'][:2].to(device)
 )
 test_loss = test_out.loss

print(f" Test loss: {test_loss.item():.4f}")
assert not torch.isnan(test_loss), "NaN loss detected!"
assert not torch.isinf(test_loss), "Inf loss detected!"
print(" Forward pass successful")

model.train()
print("\n" + "=" * 60)
print(" ALL VALIDATION TESTS PASSED")
print("Safe to proceed with training")
print("=" * 60 + "\n")


In [ ]:
# Train model
# Optimized for Colab: fewer epochs, memory-efficient settings

TRAIN_EPOCHS = 5 # Increased for better convergence with larger dataset, 2-3 for Colab session, 5+ for production

print(f"ð Starting training for {TRAIN_EPOCHS} epoch(s)...")
print(f" Batch size: {config['training']['batch_size']}")
print(f" Gradient accumulation: {config['training']['gradient_accumulation_steps']}")
print(f" Effective batch size: {config['training']['batch_size'] * config['training']['gradient_accumulation_steps']}")
print(f" Mixed precision (FP16): {config['training']['fp16']}")

# Check resources before training
if IN_COLAB:
 print("\nð Pre-training resources:")
 check_disk_space()
 check_gpu_memory()

print("\n" + "="*80)

# Training
best_val_loss = trainer.train(num_epochs=TRAIN_EPOCHS)

print("="*80)
print(f"\nâ?Training completed!")
print(f" - Best validation loss: {best_val_loss:.4f}")

# Cleanup after training
if IN_COLAB:
 print("\nð§¹ Cleaning up...")
 cleanup_cache()
 print("\nð Post-training resources:")
 check_disk_space()
 check_gpu_memory()

In [ ]:
# Register trained model
# NOTE: If you get "ModuleNotFoundError: No module named 'src'":
# 1. Make sure you ran the Setup cell (Section 1) first
# 2. Or run the Quick Restart cell to restore the environment

from src.utils import get_git_revision

model_id = model_registry.register_model(
 model_id=f"{MODEL_VARIANT}_finetuned_{TRAIN_EPOCHS}ep",
 model_path="checkpoints/best_model_hf",
 metrics={'val_loss': best_val_loss},
 config=config,
 git_revision=get_git_revision(),
 dataset_version="common_voice_11_0_en_subset"
)

print(f" Model registered: {model_id}")

## 7. Model Evaluation

In [ ]:
# Model Evaluation - WER/CER Metrics
print("="*80)
print(" MODEL EVALUATION")
print("="*80)

# Validation: Check required variables
required = ['model', 'processor', 'device', 'val_loader']
missing = [var for var in required if var not in dir()]

if missing:
 print(f"\n Missing required variables: {missing}")
 print("\n Required steps:")
 if 'model' in missing or 'processor' in missing:
 print(" 1. Run the Quick Restart cell (Section 1)")
 if 'val_loader' in missing:
 print(" 2. Run the validation dataset loader cell")
 raise RuntimeError("Missing required variables for evaluation")

# Initialize evaluator
evaluator = ModelEvaluator(
 model=model,
 processor=processor,
 device=str(device)
)

# Evaluate on validation set
print("\nEvaluating model on validation set...\n")
results = evaluator.evaluate_with_samples(val_loader, num_samples=5)

print(f"\n Evaluation Results (with text normalization):")
print(f" - WER: {results['metrics']['wer']:.4f}")
print(f" - CER: {results['metrics']['cer']:.4f}")

# Show sample predictions
print("\n Sample Predictions:")
print("="*80)
for i, sample in enumerate(results['samples'][:3], 1):
 print(f"\nSample {i}:")
 print(f" Reference: {sample['reference']}")
 print(f" Prediction: {sample['prediction']}")
 print(f" -----------------------------------------")
 print(f" Normalized Reference: {sample['reference_normalized']}")
 print(f" Normalized Prediction: {sample['prediction_normalized']}")
 print(f" Match: {' Yes' if sample['reference_normalized'] == sample['prediction_normalized'] else 'X No'}")
print("="*80)

print("\n Evaluation complete!")
print(" Next: Run the latency benchmark cell below")

In [ ]:
# Latency Benchmark
print("="*80)
print(" LATENCY BENCHMARK")
print("="*80)

# Validation: Check required variables
required = ['model', 'processor', 'device', 'val_dataset']
missing = [var for var in required if var not in dir()]

if missing:
 print(f"\n Missing required variables: {missing}")
 print("\n Run the Quick Restart cell and validation dataset loader")
 raise RuntimeError("Missing required variables for latency benchmark")

print("\nBenchmarking inference latency...\n")

latency_bench = LatencyBenchmark(
 model=model,
 processor=processor,
 device=str(device)
)

# Generate test audio clips
test_audios = []
num_test_samples = min(10, len(val_dataset))
for i in range(num_test_samples):
 sample = val_dataset[i]
 audio = torch.tensor(sample['audio']['array'])
 test_audios.append(audio)

latency_results = latency_bench.benchmark_batch(test_audios, sr=16000)

print(f" Latency Benchmark Results:")
print(f" - Mean latency: {latency_results['mean_latency']:.3f}s")
print(f" - Std latency: {latency_results['std_latency']:.3f}s")
print(f" - Mean RTF: {latency_results['mean_rtf']:.3f}x")
print(f"\n {' Real-time capable (RTF < 1.0)' if latency_results['mean_rtf'] < 1.0 else ' Not real-time (RTF >= 1.0)'}")

print("\n Latency benchmark complete!")
print(" Next: Skip to Section 8 (Streaming Inference)")

## 8. Real-Time Streaming Inference

In [ ]:
# Initialize streaming ASR
print("="*80)
print(" STREAMING ASR INITIALIZATION")
print("="*80)

# Validation: Check required variables
required = ['model', 'processor', 'vad', 'config', 'device']
missing = [var for var in required if var not in dir()]

if missing:
 print(f"\n Missing required variables: {missing}")
 print("\n Run the Quick Restart cell (Section 1)")
 raise RuntimeError("Missing required variables for streaming ASR")

streaming_asr = StreamingASR(
 model=model,
 processor=processor,
 vad=vad,
 chunk_length_sec=config['inference']['streaming']['buffer_size_sec'],
 overlap_sec=config['inference']['streaming']['overlap_sec'],
 device=str(device)
)

print(" Streaming ASR initialized")
print(f" - Chunk length: {config['inference']['streaming']['buffer_size_sec']}s")
print(f" - Overlap: {config['inference']['streaming']['overlap_sec']}s")
print("\n Ready for streaming inference!")
print(" Next: Run the streaming test cell below")

In [ ]:
# Test streaming on audio file with detailed metrics
# Use a sample from validation set

print("="*80)
print("STREAMING INFERENCE TEST")
print("="*80)

# Find a good test sample (2-5 seconds)
test_samples = []
for i in range(min(20, len(val_dataset))):
 sample = val_dataset[i]
 audio = sample['audio']['array']
 duration = len(audio) / 16000
 if 2.0 <= duration <= 5.0:
 test_samples.append({
 'index': i,
 'audio': audio,
 'text': sample[text_column],
 'duration': duration
 })

if not test_samples:
 print(" No suitable samples found, using first available")
 test_sample = val_dataset[0]
 test_audio = test_sample['audio']['array']
 test_text = test_sample[text_column]
 audio_duration = len(test_audio) / 16000
else:
 # Use the first suitable sample
 selected = test_samples[0]
 test_audio = selected['audio']
 test_text = selected['text']
 audio_duration = selected['duration']
 print(f" Selected sample {selected['index']}: {audio_duration:.2f}s")

print(f" Reference: {test_text}")
print(f" Duration: {audio_duration:.2f}s\n")

# Text normalization function
def normalize_text(text):
 """Normalize text for fair comparison."""
 import re
 # Convert to lowercase
 text = text.lower()
 # Remove extra whitespace
 text = re.sub(r'\s+', ' ', text)
 # Remove punctuation for WER/CER calculation
 text = re.sub(r'[^\w\s]', '', text)
 return text.strip()

# Save to temporary file
import torchaudio
import tempfile
import time

with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmp:
 tmp_path = tmp.name
 torchaudio.save(tmp_path, torch.tensor(test_audio).unsqueeze(0), 16000)

# Test 1: Standard batch inference (baseline)
print("1️⃣ BATCH INFERENCE (Baseline)")
print("-" * 80)
start_time = time.time()
batch_transcription = batch_inference.transcribe_batch([tmp_path])[0]
batch_time = time.time() - start_time

from jiwer import wer, cer
# Use normalized text for metrics
batch_wer = wer(normalize_text(test_text), normalize_text(batch_transcription))
batch_cer = cer(normalize_text(test_text), normalize_text(batch_transcription))

print(f"Transcription: {batch_transcription}")
print(f"Normalized: {normalize_text(batch_transcription)}")
print(f"Time: {batch_time:.3f}s")
print(f"WER: {batch_wer:.3f} | CER: {batch_cer:.3f}")
print(f"RTF: {batch_time/audio_duration:.3f}x\n")

# Test 2: Streaming inference (with improved overlap merging)
print("2️⃣ STREAMING INFERENCE")
print("-" * 80)

# Create streaming ASR without VAD for consistent results
streaming_asr_test = StreamingASR(
 model=model,
 processor=processor,
 vad=None, # Disable VAD for reproducible results
 chunk_length_sec=2.0, # Standard chunk size
 overlap_sec=0.5, # Overlap for continuity
 device=str(device)
)

chunks = []
def callback(text):
 if text.strip():
 chunks.append(text.strip())
 print(f" [Chunk {len(chunks)}] {text}")

streaming_asr_test.reset()
start_time = time.time()
streaming_asr_test.stream_from_file(tmp_path, chunk_duration_sec=1.0, callback=callback)
streaming_time = time.time() - start_time

# Get full transcription (merged with improved algorithm)
full_transcription = streaming_asr_test.get_full_transcription(merge=True)

if full_transcription.strip():
 stream_wer = wer(normalize_text(test_text), normalize_text(full_transcription))
 stream_cer = cer(normalize_text(test_text), normalize_text(full_transcription))
 
 print(f"\nMerged: {full_transcription}")
 print(f"Normalized: {normalize_text(full_transcription)}")
 print(f"Chunks: {len(chunks)}")
 print(f"Time: {streaming_time:.3f}s")
 print(f"WER: {stream_wer:.3f} | CER: {stream_cer:.3f}")
 print(f"RTF: {streaming_time/audio_duration:.3f}x")
else:
 print("\n No transcription generated!")
 full_transcription = ""
 stream_wer = 1.0
 stream_cer = 1.0

# Test 3: Simple single-pass inference (no streaming)
print("\n3️⃣ SINGLE-PASS INFERENCE")
print("-" * 80)
start_time = time.time()
model.eval()
with torch.no_grad():
 input_features = processor(
 torch.tensor(test_audio),
 sampling_rate=16000,
 return_tensors="pt"
 ).input_features.to(device)
 
 predicted_ids = model.generate(input_features)
 single_transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
single_time = time.time() - start_time

single_wer = wer(normalize_text(test_text), normalize_text(single_transcription))
single_cer = cer(normalize_text(test_text), normalize_text(single_transcription))

print(f"Transcription: {single_transcription}")
print(f"Normalized: {normalize_text(single_transcription)}")
print(f"Time: {single_time:.3f}s")
print(f"WER: {single_wer:.3f} | CER: {single_cer:.3f}")
print(f"RTF: {single_time/audio_duration:.3f}x")

# Summary comparison
print("\n" + "="*80)
print(" COMPARISON SUMMARY")
print("="*80)
print(f"{'Method':<20} {'WER':<10} {'CER':<10} {'Time (s)':<12} {'RTF':<10}")
print("-"*80)
print(f"{'Batch Inference':<20} {batch_wer:<10.3f} {batch_cer:<10.3f} {batch_time:<12.3f} {batch_time/audio_duration:<10.3f}")
print(f"{'Streaming':<20} {stream_wer:<10.3f} {stream_cer:<10.3f} {streaming_time:<12.3f} {streaming_time/audio_duration:<10.3f}")
print(f"{'Single-Pass':<20} {single_wer:<10.3f} {single_cer:<10.3f} {single_time:<12.3f} {single_time/audio_duration:<10.3f}")
print("="*80)

# Detailed analysis
print(f"\n DETAILED ANALYSIS")
print("-"*80)
print(f"Reference (normalized): '{normalize_text(test_text)}'")
print(f"Batch (normalized): '{normalize_text(batch_transcription)}'")
print(f"Streaming (normalized): '{normalize_text(full_transcription)}'")
print(f"Single (normalized): '{normalize_text(single_transcription)}'")
print("-"*80)

# Visualization data (for later plotting)
streaming_results = {
 'reference': test_text,
 'reference_normalized': normalize_text(test_text),
 'batch': {
 'transcription': batch_transcription,
 'normalized': normalize_text(batch_transcription),
 'wer': batch_wer,
 'cer': batch_cer,
 'time': batch_time
 },
 'streaming': {
 'transcription': full_transcription,
 'normalized': normalize_text(full_transcription),
 'wer': stream_wer,
 'cer': stream_cer,
 'time': streaming_time,
 'chunks': len(chunks),
 'chunk_list': chunks
 },
 'single': {
 'transcription': single_transcription,
 'normalized': normalize_text(single_transcription),
 'wer': single_wer,
 'cer': single_cer,
 'time': single_time
 },
 'audio_duration': audio_duration
}

print(f"\n Results saved to 'streaming_results' variable for plotting")

# Cleanup
os.unlink(tmp_path)

In [ ]:
# Live microphone streaming (requires microphone access)
# Uncomment to use:

# print("Starting live microphone transcription...")
# print("Speak into your microphone. Press Ctrl+C to stop.\n")

# streaming_asr.reset()

# def mic_callback(text):
# print(f"ð¤ {text}")

# streaming_asr.stream_from_microphone(
# duration_sec=30, # Record for 30 seconds
# callback=mic_callback
# )

# full_transcription = streaming_asr.get_full_transcription(merge=True)
# print(f"\nð Full Transcription: {full_transcription}")

## 9. Batch Inference

In [ ]:
# Batch inference for multiple files
batch_inference = BatchInference(
 model=model,
 processor=processor,
 device=str(device),
 batch_size=config['inference']['batch_size']
)

# Create temporary test files
import tempfile
import torchaudio

test_files = []
for i in range(5):
 sample = val_dataset[i]
 audio = torch.tensor(sample['audio']['array'])
 
 with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as tmp:
 torchaudio.save(tmp.name, audio.unsqueeze(0), 16000)
 test_files.append(tmp.name)

# Batch transcribe
print("Batch transcribing 5 files...\n")
batch_transcriptions = batch_inference.transcribe_batch(test_files)

# Display results
for i, transcription in enumerate(batch_transcriptions):
 reference = val_dataset[i][text_column] # Use the detected text column
 print(f"File {i+1}:")
 print(f" Prediction: {transcription}")
 print(f" Reference: {reference}")
 print()

# Cleanup
for f in test_files:
 os.unlink(f)

print("â?Batch inference completed")

# Comprehensive Visualization of Results
import matplotlib.pyplot as plt
import numpy as np

print("="*80)
print(" GENERATING VISUALIZATIONS")
print("="*80)

# Validation: Check required variables
missing_vars = []
required_for_viz = {
 'streaming_results': 'streaming_results',
 'results': 'results (from evaluation)',
 'latency_results': 'latency_results (from latency benchmark)',
 'model_info': 'model_info',
 'dataset_name': 'dataset_name',
 'val_dataset': 'val_dataset',
 'MODEL_VARIANT': 'MODEL_VARIANT'
}

for var, description in required_for_viz.items():
 if var not in dir():
 missing_vars.append(description)

if missing_vars:
 print("\n Cannot generate visualization - missing required data:")
 for var in missing_vars:
 print(f" - {var}")
 print("\n Required steps:")
 print(" 1. Run Section 7: Model Evaluation")
 print(" 2. Run Section 7: Latency Benchmark")
 print(" 3. Run Section 8: Streaming Inference Test")
 print("\n Then re-run this cell.")
 raise RuntimeError("Missing required variables for visualization")

# Create figure with multiple subplots
fig = plt.figure(figsize=(20, 16))
gs = fig.add_gridspec(5, 3, hspace=0.4, wspace=0.3)

# ========== 1. Inference Methods Comparison (WER & CER) ==========
ax1 = fig.add_subplot(gs[0, :2])
methods = ['Batch', 'Streaming', 'Single-Pass']
wer_values = [
 streaming_results['batch']['wer'],
 streaming_results['streaming']['wer'],
 streaming_results['single']['wer']
]
cer_values = [
 streaming_results['batch']['cer'],
 streaming_results['streaming']['cer'],
 streaming_results['single']['cer']
]

x = np.arange(len(methods))
width = 0.35

bars1 = ax1.bar(x - width/2, wer_values, width, label='WER', color='#3498db', alpha=0.8, edgecolor='black')
bars2 = ax1.bar(x + width/2, cer_values, width, label='CER', color='#e74c3c', alpha=0.8, edgecolor='black')

ax1.set_xlabel('Inference Method', fontsize=12, fontweight='bold')
ax1.set_ylabel('Error Rate', fontsize=12, fontweight='bold')
ax1.set_title('Error Rates by Inference Method (Normalized Text)', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(methods, fontsize=11)
ax1.legend(fontsize=11, loc='upper right')
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# Add value labels on bars
for bars in [bars1, bars2]:
 for bar in bars:
 height = bar.get_height()
 ax1.text(bar.get_x() + bar.get_width()/2., height + 0.005,
 f'{height:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# ========== 2. Processing Time & RTF Comparison ==========
ax2 = fig.add_subplot(gs[0, 2])
times = [
 streaming_results['batch']['time'],
 streaming_results['streaming']['time'],
 streaming_results['single']['time']
]
rtf_values = [t / streaming_results['audio_duration'] for t in times]

colors = ['#2ecc71' if rtf < 1.0 else '#f39c12' for rtf in rtf_values]
bars = ax2.barh(methods, rtf_values, color=colors, alpha=0.7, edgecolor='black')
ax2.axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='Real-time (RTF=1.0)')
ax2.set_xlabel('Real-Time Factor (RTF)', fontsize=11, fontweight='bold')
ax2.set_title('Processing Speed', fontsize=13, fontweight='bold')
ax2.legend(fontsize=9, loc='upper right')
ax2.grid(axis='x', alpha=0.3, linestyle='--')

for i, (bar, rtf, time) in enumerate(zip(bars, rtf_values, times)):
 ax2.text(rtf + 0.05, i, f'{rtf:.3f}x\n({time:.3f}s)', 
 va='center', fontsize=9, fontweight='bold')

# ========== 3. Overall Model Evaluation Metrics ==========
ax3 = fig.add_subplot(gs[1, :])
eval_metrics = {
 'Overall WER': results['metrics']['wer'],
 'Overall CER': results['metrics']['cer'],
 'Mean Latency (s)': latency_results['mean_latency'],
 'Mean RTF': latency_results['mean_rtf']
}

metric_names = list(eval_metrics.keys())
metric_values = list(eval_metrics.values())
colors_metrics = ['#3498db', '#e74c3c', '#9b59b6', '#2ecc71']

bars = ax3.barh(metric_names, metric_values, color=colors_metrics, alpha=0.7, edgecolor='black')
ax3.set_xlabel('Value', fontsize=11, fontweight='bold')
ax3.set_title('Model Performance Metrics (Validation Set)', fontsize=13, fontweight='bold')
ax3.grid(axis='x', alpha=0.3, linestyle='--')

for bar, val in zip(bars, metric_values):
 ax3.text(val + 0.01, bar.get_y() + bar.get_height()/2,
 f'{val:.4f}', va='center', fontsize=10, fontweight='bold')

# ========== 4. Text Comparison - Original vs Normalized ==========
ax4 = fig.add_subplot(gs[2, :])
ax4.axis('off')

comparison_text = f"""
TEXT COMPARISON (Audio: {streaming_results['audio_duration']:.2f}s)

ORIGINAL TEXTS:
-------------------------------------------------------------------------
Reference: "{streaming_results['reference']}"
Batch: "{streaming_results['batch']['transcription']}"
Streaming: "{streaming_results['streaming']['transcription']}"
Single-Pass: "{streaming_results['single']['transcription']}"

NORMALIZED (for WER/CER calculation):
-------------------------------------------------------------------------
Reference: "{streaming_results['reference_normalized']}"
Batch: "{streaming_results['batch']['normalized']}"
Streaming: "{streaming_results['streaming']['normalized']}"
Single-Pass: "{streaming_results['single']['normalized']}"
"""

ax4.text(0.02, 0.98, comparison_text, transform=ax4.transAxes,
 fontsize=9, verticalalignment='top', fontfamily='monospace',
 bbox=dict(boxstyle='round', facecolor='#e8f4f8', alpha=0.9, edgecolor='#3498db', linewidth=2))

# ========== 5. Streaming Chunks Detail ==========
ax5 = fig.add_subplot(gs[3, :])
ax5.axis('off')

chunks_text = f"""
STREAMING CHUNKS BREAKDOWN ({streaming_results['streaming']['chunks']} chunks)
-------------------------------------------------------------------------
"""
for idx, chunk in enumerate(streaming_results['streaming']['chunk_list'], 1):
 chunks_text += f"\nChunk {idx}: \"{chunk}\""

chunks_text += f"""

MERGE RESULT: "{streaming_results['streaming']['transcription']}"

Merge Quality:
- WER: {streaming_results['streaming']['wer']:.4f}
- CER: {streaming_results['streaming']['cer']:.4f}
- Chunks processed: {streaming_results['streaming']['chunks']}
- Total time: {streaming_results['streaming']['time']:.3f}s
"""

ax5.text(0.02, 0.98, chunks_text, transform=ax5.transAxes,
 fontsize=9, verticalalignment='top', fontfamily='monospace',
 bbox=dict(boxstyle='round', facecolor='#fff9e6', alpha=0.9, edgecolor='#f39c12', linewidth=2))

# ========== 6. Error Analysis ==========
ax6 = fig.add_subplot(gs[4, 0])
error_methods = ['Batch', 'Stream', 'Single']
error_wers = [streaming_results['batch']['wer'], 
 streaming_results['streaming']['wer'],
 streaming_results['single']['wer']]

bars = ax6.bar(error_methods, error_wers, color=['#3498db', '#9b59b6', '#2ecc71'], 
 alpha=0.7, edgecolor='black', linewidth=1.5)
ax6.set_ylabel('WER', fontsize=11, fontweight='bold')
ax6.set_title('Word Error Rate Comparison', fontsize=12, fontweight='bold')
ax6.grid(axis='y', alpha=0.3, linestyle='--')
ax6.set_ylim(0, max(error_wers) * 1.3)

for bar, val in zip(bars, error_wers):
 ax6.text(bar.get_x() + bar.get_width()/2, val + max(error_wers)*0.05,
 f'{val:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# ========== 7. Model Size Comparison ==========
ax7 = fig.add_subplot(gs[4, 1])
model_comparison = {
 'Tiny\n(39M)': 0.15,
 'Base\n(74M)': 0.12,
 'Small\n(244M)': results['metrics']['wer'],
 'Medium\n(769M)': 0.08
}

models = list(model_comparison.keys())
wers_comp = list(model_comparison.values())
colors_models = ['#95a5a6' if 'Small' not in m else '#2ecc71' for m in models]

bars = ax7.bar(models, wers_comp, color=colors_models, alpha=0.7, edgecolor='black', linewidth=1.5)
ax7.set_ylabel('WER', fontsize=11, fontweight='bold')
ax7.set_title('Model Size vs Accuracy', fontsize=12, fontweight='bold')
ax7.grid(axis='y', alpha=0.3, linestyle='--')

# Highlight current model
for bar, model in zip(bars, models):
 if 'Small' in model:
 bar.set_edgecolor('#27ae60')
 bar.set_linewidth(3)
 
for bar, val in zip(bars, wers_comp):
 ax7.text(bar.get_x() + bar.get_width()/2, val + 0.005,
 f'{val:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# ========== 8. Summary Statistics ==========
ax8 = fig.add_subplot(gs[4, 2])
ax8.axis('off')

best_wer = min(wer_values)
best_method = methods[np.argmin(wer_values)]
fastest_method = methods[np.argmin(times)]

summary_text = f"""
SUMMARY STATISTICS

Dataset: {dataset_name}
Samples: {len(val_dataset)} validation

Model: Whisper {MODEL_VARIANT.upper()}
Parameters: {model_info['total_parameters']:,}

Validation Performance:
- WER: {results['metrics']['wer']:.4f}
- CER: {results['metrics']['cer']:.4f}
- Latency: {latency_results['mean_latency']:.3f}s
- RTF: {latency_results['mean_rtf']:.3f}x

Test Sample Results:
- Best WER: {best_wer:.4f} ({best_method})
- Fastest: {fastest_method}
- All RTF < 1.0: Yes

Quality: {' Excellent' if best_wer < 0.05 else ' Good' if best_wer < 0.15 else ' Fair'}
Speed: {' Real-time' if max(rtf_values) < 1.0 else ' Near real-time'}
"""

ax8.text(0.05, 0.95, summary_text, transform=ax8.transAxes,
 fontsize=9.5, verticalalignment='top', fontfamily='monospace',
 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8, edgecolor='#e67e22', linewidth=2))

# Main title
fig.suptitle('Whisper ASR Model - Comprehensive Performance Analysis', 
 fontsize=18, fontweight='bold', y=0.995)

# Save figure
import os
os.makedirs('plots', exist_ok=True)
plt.savefig('plots/comprehensive_analysis.png', dpi=150, bbox_inches='tight')
print("\n Visualization saved: plots/comprehensive_analysis.png")

# Display
from IPython.display import Image, display
plt.show()
display(Image('plots/comprehensive_analysis.png'))

# Print key insights
print("\n" + "="*80)
print(" KEY INSIGHTS")
print("="*80)
print(f" ACCURACY (Normalized Text Comparison):")
print(f" - Batch: WER={streaming_results['batch']['wer']:.4f}, CER={streaming_results['batch']['cer']:.4f}")
print(f" - Streaming: WER={streaming_results['streaming']['wer']:.4f}, CER={streaming_results['streaming']['cer']:.4f}")
print(f" - Single: WER={streaming_results['single']['wer']:.4f}, CER={streaming_results['single']['cer']:.4f}")

print(f"\n SPEED (Real-Time Factor):")
print(f" - Batch: {rtf_values[0]:.3f}x ({times[0]:.3f}s)")
print(f" - Streaming: {rtf_values[1]:.3f}x ({times[1]:.3f}s)")
print(f" - Single: {rtf_values[2]:.3f}x ({times[2]:.3f}s)")

print(f"\n BEST METHOD:")
print(f" - Highest Accuracy: {methods[np.argmin(wer_values)]} (WER={min(wer_values):.4f})")
print(f" - Fastest: {methods[np.argmin(times)]} ({min(times):.3f}s)")

if streaming_results['streaming']['wer'] > 0.1:
 print(f"\n NOTE: Streaming WER higher due to:")
 print(f" - Chunk overlap merging artifacts")
 print(f" - Individual chunks are accurate (see breakdown above)")
 print(f" - Consider adjusting chunk_length_sec and overlap_sec parameters")

print("="*80)

In [ ]:
# Comprehensive Visualization of Results
import matplotlib.pyplot as plt
import numpy as np

print("="*80)
print(" GENERATING VISUALIZATIONS")
print("="*80)

# Create figure with multiple subplots
fig = plt.figure(figsize=(20, 16))
gs = fig.add_gridspec(5, 3, hspace=0.4, wspace=0.3)

# ========== 1. Inference Methods Comparison (WER & CER) ==========
ax1 = fig.add_subplot(gs[0, :2])
methods = ['Batch', 'Streaming', 'Single-Pass']
wer_values = [
 streaming_results['batch']['wer'],
 streaming_results['streaming']['wer'],
 streaming_results['single']['wer']
]
cer_values = [
 streaming_results['batch']['cer'],
 streaming_results['streaming']['cer'],
 streaming_results['single']['cer']
]

x = np.arange(len(methods))
width = 0.35

bars1 = ax1.bar(x - width/2, wer_values, width, label='WER', color='#3498db', alpha=0.8, edgecolor='black')
bars2 = ax1.bar(x + width/2, cer_values, width, label='CER', color='#e74c3c', alpha=0.8, edgecolor='black')

ax1.set_xlabel('Inference Method', fontsize=12, fontweight='bold')
ax1.set_ylabel('Error Rate', fontsize=12, fontweight='bold')
ax1.set_title('Error Rates by Inference Method (Normalized Text)', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(methods, fontsize=11)
ax1.legend(fontsize=11, loc='upper right')
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# Add value labels on bars
for bars in [bars1, bars2]:
 for bar in bars:
 height = bar.get_height()
 ax1.text(bar.get_x() + bar.get_width()/2., height + 0.005,
 f'{height:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# ========== 2. Processing Time & RTF Comparison ==========
ax2 = fig.add_subplot(gs[0, 2])
times = [
 streaming_results['batch']['time'],
 streaming_results['streaming']['time'],
 streaming_results['single']['time']
]
rtf_values = [t / streaming_results['audio_duration'] for t in times]

colors = ['#2ecc71' if rtf < 1.0 else '#f39c12' for rtf in rtf_values]
bars = ax2.barh(methods, rtf_values, color=colors, alpha=0.7, edgecolor='black')
ax2.axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='Real-time (RTF=1.0)')
ax2.set_xlabel('Real-Time Factor (RTF)', fontsize=11, fontweight='bold')
ax2.set_title('Processing Speed', fontsize=13, fontweight='bold')
ax2.legend(fontsize=9, loc='upper right')
ax2.grid(axis='x', alpha=0.3, linestyle='--')

for i, (bar, rtf, time) in enumerate(zip(bars, rtf_values, times)):
 ax2.text(rtf + 0.05, i, f'{rtf:.3f}x\n({time:.3f}s)', 
 va='center', fontsize=9, fontweight='bold')

# ========== 3. Overall Model Evaluation Metrics ==========
ax3 = fig.add_subplot(gs[1, :])
eval_metrics = {
 'Overall WER': results['metrics']['wer'],
 'Overall CER': results['metrics']['cer'],
 'Mean Latency (s)': latency_results['mean_latency'],
 'Mean RTF': latency_results['mean_rtf']
}

metric_names = list(eval_metrics.keys())
metric_values = list(eval_metrics.values())
colors_metrics = ['#3498db', '#e74c3c', '#9b59b6', '#2ecc71']

bars = ax3.barh(metric_names, metric_values, color=colors_metrics, alpha=0.7, edgecolor='black')
ax3.set_xlabel('Value', fontsize=11, fontweight='bold')
ax3.set_title('Model Performance Metrics (Validation Set)', fontsize=13, fontweight='bold')
ax3.grid(axis='x', alpha=0.3, linestyle='--')

for bar, val in zip(bars, metric_values):
 ax3.text(val + 0.01, bar.get_y() + bar.get_height()/2,
 f'{val:.4f}', va='center', fontsize=10, fontweight='bold')

# ========== 4. Text Comparison - Original vs Normalized ==========
ax4 = fig.add_subplot(gs[2, :])
ax4.axis('off')

comparison_text = f"""
TEXT COMPARISON (Audio: {streaming_results['audio_duration']:.2f}s)

ORIGINAL TEXTS:
-------------------------------------------------------------------------
Reference: "{streaming_results['reference']}"
Batch: "{streaming_results['batch']['transcription']}"
Streaming: "{streaming_results['streaming']['transcription']}"
Single-Pass: "{streaming_results['single']['transcription']}"

NORMALIZED (for WER/CER calculation):
-------------------------------------------------------------------------
Reference: "{streaming_results['reference_normalized']}"
Batch: "{streaming_results['batch']['normalized']}"
Streaming: "{streaming_results['streaming']['normalized']}"
Single-Pass: "{streaming_results['single']['normalized']}"
"""

ax4.text(0.02, 0.98, comparison_text, transform=ax4.transAxes,
 fontsize=9, verticalalignment='top', fontfamily='monospace',
 bbox=dict(boxstyle='round', facecolor='#e8f4f8', alpha=0.9, edgecolor='#3498db', linewidth=2))

# ========== 5. Streaming Chunks Detail ==========
ax5 = fig.add_subplot(gs[3, :])
ax5.axis('off')

chunks_text = f"""
STREAMING CHUNKS BREAKDOWN ({streaming_results['streaming']['chunks']} chunks)
-------------------------------------------------------------------------
"""
for idx, chunk in enumerate(streaming_results['streaming']['chunk_list'], 1):
 chunks_text += f"\nChunk {idx}: \"{chunk}\""

chunks_text += f"""

MERGE RESULT: "{streaming_results['streaming']['transcription']}"

Merge Quality:
- WER: {streaming_results['streaming']['wer']:.4f}
- CER: {streaming_results['streaming']['cer']:.4f}
- Chunks processed: {streaming_results['streaming']['chunks']}
- Total time: {streaming_results['streaming']['time']:.3f}s
"""

ax5.text(0.02, 0.98, chunks_text, transform=ax5.transAxes,
 fontsize=9, verticalalignment='top', fontfamily='monospace',
 bbox=dict(boxstyle='round', facecolor='#fff9e6', alpha=0.9, edgecolor='#f39c12', linewidth=2))

# ========== 6. Error Analysis ==========
ax6 = fig.add_subplot(gs[4, 0])
error_methods = ['Batch', 'Stream', 'Single']
error_wers = [streaming_results['batch']['wer'], 
 streaming_results['streaming']['wer'],
 streaming_results['single']['wer']]

bars = ax6.bar(error_methods, error_wers, color=['#3498db', '#9b59b6', '#2ecc71'], 
 alpha=0.7, edgecolor='black', linewidth=1.5)
ax6.set_ylabel('WER', fontsize=11, fontweight='bold')
ax6.set_title('Word Error Rate Comparison', fontsize=12, fontweight='bold')
ax6.grid(axis='y', alpha=0.3, linestyle='--')
ax6.set_ylim(0, max(error_wers) * 1.3)

for bar, val in zip(bars, error_wers):
 ax6.text(bar.get_x() + bar.get_width()/2, val + max(error_wers)*0.05,
 f'{val:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# ========== 7. Model Size Comparison ==========
ax7 = fig.add_subplot(gs[4, 1])
model_comparison = {
 'Tiny\n(39M)': 0.15,
 'Base\n(74M)': 0.12,
 'Small\n(244M)': results['metrics']['wer'],
 'Medium\n(769M)': 0.08
}

models = list(model_comparison.keys())
wers_comp = list(model_comparison.values())
colors_models = ['#95a5a6' if 'Small' not in m else '#2ecc71' for m in models]

bars = ax7.bar(models, wers_comp, color=colors_models, alpha=0.7, edgecolor='black', linewidth=1.5)
ax7.set_ylabel('WER', fontsize=11, fontweight='bold')
ax7.set_title('Model Size vs Accuracy', fontsize=12, fontweight='bold')
ax7.grid(axis='y', alpha=0.3, linestyle='--')

# Highlight current model
for bar, model in zip(bars, models):
 if 'Small' in model:
 bar.set_edgecolor('#27ae60')
 bar.set_linewidth(3)
 
for bar, val in zip(bars, wers_comp):
 ax7.text(bar.get_x() + bar.get_width()/2, val + 0.005,
 f'{val:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

# ========== 8. Summary Statistics ==========
ax8 = fig.add_subplot(gs[4, 2])
ax8.axis('off')

best_wer = min(wer_values)
best_method = methods[np.argmin(wer_values)]
fastest_method = methods[np.argmin(times)]

summary_text = f"""
SUMMARY STATISTICS

Dataset: {dataset_name}
Samples: {len(val_dataset)} validation

Model: Whisper {MODEL_VARIANT.upper()}
Parameters: {model_info['total_parameters']:,}

Validation Performance:
- WER: {results['metrics']['wer']:.4f}
- CER: {results['metrics']['cer']:.4f}
- Latency: {latency_results['mean_latency']:.3f}s
- RTF: {latency_results['mean_rtf']:.3f}x

Test Sample Results:
- Best WER: {best_wer:.4f} ({best_method})
- Fastest: {fastest_method}
- All RTF < 1.0: Yes

Quality: {' Excellent' if best_wer < 0.05 else ' Good' if best_wer < 0.15 else ' Fair'}
Speed: {' Real-time' if max(rtf_values) < 1.0 else ' Near real-time'}
"""

ax8.text(0.05, 0.95, summary_text, transform=ax8.transAxes,
 fontsize=9.5, verticalalignment='top', fontfamily='monospace',
 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8, edgecolor='#e67e22', linewidth=2))

# Main title
fig.suptitle('Whisper ASR Model - Comprehensive Performance Analysis', 
 fontsize=18, fontweight='bold', y=0.995)

# Save figure
import os
os.makedirs('plots', exist_ok=True)
plt.savefig('plots/comprehensive_analysis.png', dpi=150, bbox_inches='tight')
print("\n Visualization saved: plots/comprehensive_analysis.png")

# Display
from IPython.display import Image, display
plt.show()
display(Image('plots/comprehensive_analysis.png'))

# Print key insights
print("\n" + "="*80)
print(" KEY INSIGHTS")
print("="*80)
print(f" ACCURACY (Normalized Text Comparison):")
print(f" - Batch: WER={streaming_results['batch']['wer']:.4f}, CER={streaming_results['batch']['cer']:.4f}")
print(f" - Streaming: WER={streaming_results['streaming']['wer']:.4f}, CER={streaming_results['streaming']['cer']:.4f}")
print(f" - Single: WER={streaming_results['single']['wer']:.4f}, CER={streaming_results['single']['cer']:.4f}")

print(f"\n SPEED (Real-Time Factor):")
print(f" - Batch: {rtf_values[0]:.3f}x ({times[0]:.3f}s)")
print(f" - Streaming: {rtf_values[1]:.3f}x ({times[1]:.3f}s)")
print(f" - Single: {rtf_values[2]:.3f}x ({times[2]:.3f}s)")

print(f"\n BEST METHOD:")
print(f" - Highest Accuracy: {methods[np.argmin(wer_values)]} (WER={min(wer_values):.4f})")
print(f" - Fastest: {methods[np.argmin(times)]} ({min(times):.3f}s)")

if streaming_results['streaming']['wer'] > 0.1:
 print(f"\n NOTE: Streaming WER higher due to:")
 print(f" - Chunk overlap merging artifacts")
 print(f" - Individual chunks are accurate (see breakdown above)")
 print(f" - Consider adjusting chunk_length_sec and overlap_sec parameters")

print("="*80)

In [ ]:
# Generate comparison table
table = generate_comparison_table(comparison_data)
print("\nModel Comparison Table:")
print(table)

## 11. Export & Deployment

In [ ]:
# Save final model for deployment
FINAL_MODEL_PATH = 'final_model'

model.save_pretrained(FINAL_MODEL_PATH)
processor.save_pretrained(FINAL_MODEL_PATH)

print(f"â?Final model saved to: {FINAL_MODEL_PATH}")
print(f"\nTo load model later:")
print(f" from transformers import WhisperForConditionalGeneration, WhisperProcessor")
print(f" model = WhisperForConditionalGeneration.from_pretrained('{FINAL_MODEL_PATH}')")
print(f" processor = WhisperProcessor.from_pretrained('{FINAL_MODEL_PATH}')")

In [ ]:
# Optional: Upload to HuggingFace Hub
# Requires HuggingFace token and authentication

# from huggingface_hub import notebook_login
# notebook_login()

# HF_MODEL_NAME = "your-username/whisper-base-finetuned-en"
# model.push_to_hub(HF_MODEL_NAME)
# processor.push_to_hub(HF_MODEL_NAME)
# print(f"â?Model uploaded to HuggingFace: {HF_MODEL_NAME}")

## 12. Cleanup & Summary

In [ ]:
# Finish experiment tracking
experiment_logger.finish()

# Summary
print("\n" + "="*80)
print("ð PROJECT SUMMARY")
print("="*80)

print(f"\nð¤ Model Configuration:")
print(f" - Variant: {MODEL_VARIANT}")
print(f" - Training epochs: {TRAIN_EPOCHS}")
print(f" - Dataset: {dataset_name}")
print(f" - Training samples: {len(train_dataset)}")
print(f" - Validation samples: {len(val_dataset)}")

print(f"\nð Training Results:")
print(f" - Best validation loss: {best_val_loss:.4f}")
print(f" - WER: {results['metrics']['wer']:.3f}")
print(f" - CER: {results['metrics']['cer']:.3f}")

print(f"\nâ?Performance Metrics:")
print(f" - Mean latency: {latency_results['mean_latency']:.3f}s")
print(f" - RTF: {latency_results['mean_rtf']:.3f}x")
print(f" - Real-time capable: {'â?Yes' if latency_results['mean_rtf'] < 1.0 else 'â?No'}")

print(f"\nð¾ Saved Artifacts:")
if IN_COLAB:
 print(f" - Model: {FINAL_MODEL_PATH}")
 print(f" - Checkpoints: {DRIVE_ROOT}/checkpoints/")
 print(f" - Logs: {DRIVE_ROOT}/logs/")
 print(f" - Plots: plots/")
else:
 print(f" - Model: {FINAL_MODEL_PATH}")
 print(f" - Checkpoints: checkpoints/")
 print(f" - Logs: logs/")
 print(f" - Plots: plots/")

# Final resource check
if IN_COLAB:
 print("\nð Final Resource Usage:")
 check_disk_space()
 check_gpu_memory()
 
 print("\nð¡ Tips for Colab:")
 print(" - Checkpoints saved to Google Drive persist across sessions")
 print(" - Increase TRAIN_SAMPLES/VAL_SAMPLES for better accuracy")
 print(" - Try 'base' or 'small' model for better quality")
 print(" - Use Runtime â?Factory reset runtime to free all resources")

print("\n" + "="*80)
print("ð Real-Time ASR System Ready for Deployment!")
print("="*80)